## Generate Rockfall Matrix
<br>
  
#### Syntax
`RunPars = RockfallGenerator(RunPars)` <br>
#### Input 
`RunPars` : dictionary of size 9 with the information relevant to the run. <br>
#### Variables Used
##### Run Pars
`erosion_rate` : base erosion rate (cm yr-1) <br> 
`c` : Pareto distribution shape value <br>
`scenarios` : number of scenarios per run <br>
`total_time` : total time in the runs (yrs) <br>

#### Output
##### Run Pars
`RockfallMatrix` : an matrix of size [s,total_time] with the annual rockfall depth (cm) for stochastic erosion, as defined by a pareto distribution with a shape value c.<br>
`TrueErosionRates` : vector size s with the true erosion rates (L / total_time) (cm yr-1) <br>

#### Notes
**Date of Creation:** 1. Juli 2021 <br>
**Author:** Donovan Dennis <br>
**Update:** <br>

In [64]:
def RockfallGenerator(RunPars):    
    # bring in the relevant parameters
    erosion_rate = RunPars['erosion_rate']
    c = RunPars['c']
    scenarios = RunPars['scenarios']
    total_time = RunPars['total_time']
    
    rockfall_mat = np.empty((scenarios, total_time)) # open up a matrix for the annual rockfall sizes
    ErosionRates = [0] * scenarios # define a vector of erosion rates
    
    # calculate a scaling factor such that the mean is equivalent to the input erosion rate
    genparet_scale = erosion_rate*(1-c)  
    
    
    # loop function that finds a distribution of rockfalls such that:
    # the largest is < 1490 cm
    # the true erosion rate is not > or < than 110% or 90% of the input erosion rate
    # the minimum rockfall value every year is 0.000001 cm
    for i in range(scenarios):
        while True:
            falls = scipy.stats.genpareto.rvs(size = total_time,
                                              c = c, 
                                              scale = genparet_scale,
                                             loc = 0.000001)
            test_rate = np.sum(falls) / total_time

            if np.max(falls) > 1490.0:
                move_on = False
            elif test_rate > erosion_rate + (0.1 * erosion_rate):
                move_on = False
            elif test_rate < erosion_rate - (0.1 * erosion_rate):
                move_on = False
            else:
                move_on = True
            
            if move_on == True:
                rockfall_mat[i,:] = falls
                break
        ErosionRates[i] = test_rate
    
    # assign to the parameters dictionary
    RunPars['RockfallMatrix'] = rockfall_mat 
    RunPars['TrueErosionRates'] = ErosionRates 
    
    return RunPars